# ICAMP Model (Merton, 1973) #

### Adding a Hedge Portfolio to the CAPM model ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_financial_data
from capm_toolkit import wexp
from capm_toolkit import compute_factor_contributions
from capm_toolkit import compute_residual_returns

# Pretty Notation
from IPython.display import display, Math

In [2]:
# Import Data
ticker = 'AAPL'

# Stock Data (Adjust Ticker)
df_stock = import_financial_data(ticker)

# Get the important data for the Risk Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

# Get the important data for the S&P500
sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

# Get the important data for the Zero Beta Portfolio
zero_beta = pd.read_csv(r"..\additional_data\zero_beta_portfolio.csv")
zero_beta = zero_beta.set_index('Date')
zero_beta.index = pd.to_datetime(zero_beta.index)

In [3]:
# Create the DataFrame
data = pd.DataFrame()

# Create the Columns
data['stock_returns'] = df_stock['adj_close'].pct_change(1)
data['risk_free_rate'] = (((1 + (rfr['risk_free_rate'].div(100)))**(1/360)) - 1)
data['benchmark_returns'] = sp500['sp_500'].pct_change(1)
data['zero_beta_returns'] = zero_beta

# Create the Excess Returns
data['stock_excess_returns'] = data['stock_returns'] - data['risk_free_rate']
data['benchmark_excess_returns'] = data['benchmark_returns'] - data['risk_free_rate']
data['hedge_port_excess_returns'] = data['zero_beta_returns'] - data['risk_free_rate']

data.dropna(inplace = True)
data

In [4]:
# Check Correlations Matrix

data[['stock_excess_returns', 'benchmark_excess_returns', 'hedge_port_excess_returns']].corr()

In [5]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(data[['stock_excess_returns', 'benchmark_excess_returns', 'hedge_port_excess_returns']].cumsum(), label=data[['stock_excess_returns', 'benchmark_excess_returns', 'hedge_port_excess_returns']].columns, alpha=0.7)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [6]:
# Calculate the ICAPM with the whole time stamp

y = data['stock_excess_returns']

x = data[['benchmark_excess_returns', 'hedge_port_excess_returns']]
x = sm.add_constant(x)

# Calculate Weights
window = len(y)
weights = window * wexp(window, window/2)

#Model specification
model = sm.WLS(
    y, 
    x,
    missing='drop',
    weights=weights,
    )   
     
#the results of the model
results = model.fit() 
    
#here we check the summary
print(results.summary())   

In [7]:
# Set rolling window size
window = 252
weights = window * wexp(window, window/2)

# Lists to store rolling coefficients
betas = []
index = []
lower_bounds = []
upper_bounds = []

# Rolling regression
for i in range(window, len(data)):
    Y_window = y.iloc[i - window:i]
    X_window = x.iloc[i - window:i]
    
    X_window = sm.add_constant(X_window)

    # Fit WLS model
    model = sm.WLS(
        Y_window, 
        X_window, 
        missing='drop', 
        weights=weights,
    ).fit()

    # Store coefficients (const, X1, X2)
    betas.append(model.params.values)
    index.append(data.index[i])  # Use the last date of the window

    # Store lower and upper bounds of 95% confidence intervals
    ci = model.conf_int(alpha=0.05)  # 95% CI
    lower_bounds.append(ci.iloc[:, 0].values)  # First column: lower bound
    upper_bounds.append(ci.iloc[:, 1].values)  # Second column: upper bound


In [8]:
# Convert list of coefficients to DataFrame
betas_df = pd.DataFrame(betas, columns=x.columns, index=index)

betas_df

In [9]:
# Lower bounds DataFrame
lower_df = pd.DataFrame(lower_bounds, columns=[f'{col}_lower' for col in x.columns], index=index)

lower_df

In [10]:
# Upper bounds DataFrame
upper_df = pd.DataFrame(upper_bounds, columns=[f'{col}_upper' for col in x.columns], index=index)

upper_df

In [11]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['benchmark_excess_returns'], label='Market Beta', color='orange', alpha=0.7)
plt.plot(betas_df['hedge_port_excess_returns'], label='Hedge Portfolio Beta', color='blue', alpha=0.7)
# plt.plot(betas_df['const'], label='Alpha', color='green', alpha=0.7)

# Config
plt.title('Beta vs Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

In [12]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Market Beta
betas_df['benchmark_excess_returns'].plot(color = 'blue', ax = ax1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Market Beta', 
    color='blue'
    )

# Hedge Portfolio Beta
ax2 = ax1.twinx()

betas_df['hedge_port_excess_returns'].plot(color = 'orange', ax = ax2, alpha=0.7)
ax2.set_ylabel(
    'Hedge Portfolio Beta', 
    color='orange'
    )

plt.title('Beta vs Beta Time Series')
plt.show()

In [13]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['benchmark_excess_returns'], label='Market Beta', color='black', alpha=0.7)
plt.fill_between(upper_df.index, lower_df['benchmark_excess_returns_lower'], upper_df['benchmark_excess_returns_upper'], color='skyblue', alpha=0.2, label='95% CI')

# Config
plt.title('Market Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

In [14]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['hedge_port_excess_returns'], label='Hedge Portfolio Beta', color='black', alpha=0.7)
plt.fill_between(upper_df.index, lower_df['hedge_port_excess_returns_lower'], upper_df['hedge_port_excess_returns_upper'], color='peachpuff', alpha=0.5, label='95% CI')
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Hedge Portfolio Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

In [15]:
hedge_beta_bounds_df = pd.DataFrame(index=betas_df.index)
hedge_beta_bounds_df['beta'] = betas_df['hedge_port_excess_returns']
hedge_beta_bounds_df['lower'] = lower_df['hedge_port_excess_returns_lower']
hedge_beta_bounds_df['upper'] = upper_df['hedge_port_excess_returns_upper']
hedge_beta_bounds_df['real_beta'] = hedge_beta_bounds_df.apply(
    lambda row: row['beta'] if (row['lower'] > 0 or row['upper'] < 0) else 0,
    axis=1
)

In [16]:
plt.figure(figsize=(10, 6))
plt.plot(hedge_beta_bounds_df['real_beta'], label='Hedge Portfolio Beta', color='black', alpha=0.7)

# Config
plt.title('Hedge Portfolio Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

### Adding Constrains to the Model ###

In [17]:
# Remembering the Form of the Constrained OLS

display(Math(r"\beta=(X^⊤X)^{-1}(X^⊤Y)-P"))
display(Math(r"P=\frac{R^⊤(X^⊤X)^{-1}(X^⊤Y)-q}{R^⊤(X^⊤X)^{-1}R}(X^⊤X)^{-1}R"))

In [18]:
#Show the betas

model_betas = results.params

model_betas

In [19]:
# Make the Models using GLM
OLS_from_GLM = sm.GLM(y, x)

result_c = OLS_from_GLM.fit_constrained('benchmark_excess_returns + hedge_port_excess_returns = 1')

#here we check the summary
print(result_c.summary())   

In [20]:
# Create the Residual Returns DF

icapm_returns_df = pd.DataFrame(index=betas_df.index)

icapm_returns_df['stock_excess_returns'] = data['stock_excess_returns']
icapm_returns_df['benchmark_excess_returns'] = data['benchmark_excess_returns']
icapm_returns_df['hedge_port_excess_returns'] = data['hedge_port_excess_returns']

icapm_returns_df['factor_market_returns'] = compute_factor_contributions(
    icapm_returns_df['benchmark_excess_returns'],
    betas_df['benchmark_excess_returns'],
)
icapm_returns_df['factor_hedge_returns'] = compute_factor_contributions(
    icapm_returns_df['hedge_port_excess_returns'],
    betas_df['hedge_port_excess_returns'],
)

icapm_returns_df['residual_returns'] = compute_residual_returns(
    icapm_returns_df['stock_excess_returns'],
    icapm_returns_df[['benchmark_excess_returns', 'hedge_port_excess_returns']],
    betas_df[['benchmark_excess_returns', 'hedge_port_excess_returns']],
)

icapm_returns_df['model_returns'] = icapm_returns_df['factor_market_returns'] + icapm_returns_df['factor_hedge_returns']

icapm_returns_df

In [22]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(icapm_returns_df['stock_excess_returns'].cumsum(), label='Stock Returns', alpha=1)
plt.plot(icapm_returns_df['factor_market_returns'].cumsum(), label='Market Returns', alpha=1)
plt.plot(icapm_returns_df['factor_hedge_returns'].cumsum(), label='Hedge Returns', alpha=1)
plt.plot(icapm_returns_df['residual_returns'].cumsum(), label='Residual Returns', alpha=1)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.show()